In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3,4,5,6,7"
# os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'upb'

In [ ]:
os.environ['HF_HOME'] = './models'
print(f"Current cache directory: {os.environ['HF_HOME']}")

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
from transformers import pipeline

device = "cuda"

In [4]:
model = "dbrx-instruct"

In [ ]:
# pipe = pipeline("text-generation", model="databricks/{}".format(model), max_length=2048, device=device)

tokenizer = AutoTokenizer.from_pretrained("databricks/{}".format(model), token="")
llm = AutoModelForCausalLM.from_pretrained("databricks/{}".format(model), device_map="auto", torch_dtype=torch.bfloat16, token="")

Loading checkpoint shards:   0%|          | 0/61 [00:00<?, ?it/s]

test stage

In [6]:
import json
from tqdm import tqdm

In [7]:
def convert_unicode_to_plain_text(unicode_str):
    if isinstance(unicode_str, str) and '\\u' in unicode_str:
        plain_text = unicode_str.encode().decode('unicode_escape')
        return plain_text
    else:
        return unicode_str

In [8]:
template = "Give me a sense of your preference regarding the following question or statement. Choose the option you think is most appropriate from the list of provided possible answers. \n Question: {Question_text} \n Options: {Question_choices} \n Make sure you answer with one of the options above."

In [9]:
para_dict = [{
    "role": "user",
    "content": ""
    }]

para_dict2 = [
    {
        "role": "user",
        "content": "Some people have a political issue that they care about more than most other issues. They might think about the issue a lot. They might pay particular attention to news about that issue, even when it's not making national news. They might focus on what political candidates say about that issue, and decide who to vote for on the basis of that issue. Or they might just care about the issue a lot. Is there an issue like that for you?"
    },
    {"role": "assistant",
        "content": "Yes"}, 
    {"role": "user",
        "content":  ""}
    ]

In [ ]:
root = "./question_json_ver3"
script_list = os.listdir(root)
script_list.sort()

In [15]:
if not os.path.exists(model): os.mkdir(model)

for script in tqdm(script_list):
    if not os.path.exists(os.path.join(model, script)):
# if 1:
#         script = script_list[0]
        print("working on {} ...".format(script))

        with open(os.path.join(root, script), 'r') as f:
            data = json.load(f)
        for i, question in tqdm(enumerate(data['questions'])):
            execute = 0
            if "Answer_text" not in question.keys():
                q, c = question["Question_text"], question["Question_choices"]
                if len(c) > 0:
                    execute = 1
                    template_tmp = template.format(**({"Question_text":q, "Question_choices":c}))

                    para_tmp = para_dict.copy()
                    para_tmp[-1]["content"] = template_tmp
                elif len(c) == 0 and ("yes" in data['questions'][i-1]["Answer_text"].lower()):
                    execute = 1
                    template_tmp = q

                    para_tmp = para_dict2.copy()
                    para_tmp[-1]["content"] = template_tmp
                else:
                    continue

                if execute:
                    # try:
                            messages = para_tmp
                            input_ids = tokenizer.apply_chat_template(messages, return_dict=True, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

                            output = llm.generate(**input_ids, max_new_tokens=512, pad_token_id=tokenizer.eos_token_id)
                            output = tokenizer.decode(output[0])
                    # except:
                    #         response = "#ERROR"
                            data['questions'][i]["Answer_text"] = convert_unicode_to_plain_text(output)
                    
                with open(os.path.join(model, script), 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=4)
            

  0%|          | 0/10 [00:00<?, ?it/s]

working on questions_json_1.json ...


45it [19:00, 25.35s/it]
 10%|█         | 1/10 [19:00<2:51:06, 1140.68s/it]

working on questions_json_10.json ...
